In [1]:
%pip install -qU fastapi uvicorn aiortc

Note: you may need to restart the kernel to use updated packages.


To make fastapi run in jupyter

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
from aiortc import RTCPeerConnection, RTCSessionDescription
from aiortc.contrib.media import MediaPlayer, MediaRelay

In [ ]:
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],  # Allow all HTTP methods
    allow_headers=["*"]   # Allow all headers
)

relay = MediaRelay()
pc = RTCPeerConnection()

In [5]:
@app.post("/api/webrtc/offer")
async def webrtc_offer(request: Request):
    data = await request.json()
    offer = data.get("sdp")

    # Set the remote SDP (offer)
    await pc.setRemoteDescription(RTCSessionDescription(sdp=offer, type="offer"))

    # Add media processing logic here if needed
    @pc.on("track")
    async def on_track(track):
        print(f"Received track: {track.kind}")
        if track.kind == "audio":
            player = MediaPlayer(track)
            relay.subscribe(player)

    # Create an SDP answer
    answer = await pc.createAnswer()
    await pc.setLocalDescription(answer)

    return {"answer": pc.localDescription.sdp}

In [6]:
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [640089]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:38748 - "OPTIONS /api/webrtc/offer HTTP/1.1" 200 OK
INFO:     127.0.0.1:38748 - "POST /api/webrtc/offer HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [640089]
